In [1]:
import pandas as pd

In [2]:
npr = pd.read_csv("C:\\Users\\fabia\\Desktop\\modelos\\NewsF.csv",sep=";")

In [3]:
npr.head()

,Unnamed: 0,Type,News
0,0,Business,Ad sales boost Time Warner profit\n\nQuarterly...
1,1,Business,Dollar gains on Greenspan speech\n\nThe dollar...
2,2,Business,Yukos unit buyer faces loan claim\n\nThe owner...
3,3,Business,High fuel prices hit BA's profits\n\nBritish A...
4,4,Business,Pernod takeover talk lifts Domecq\n\nShares in...


# Preprocesamiento

In [4]:
#Vectorizacion
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_df=0.9,min_df=2,stop_words='english')
dtm = cv.fit_transform(npr['News'])

In [6]:
dtm

<2225x17473 sparse matrix of type '<class 'numpy.int64'>'
	with 310493 stored elements in Compressed Sparse Row format>

# TruncatedSVD Transformacion

In [7]:
from sklearn.decomposition import TruncatedSVD
tsvd_model = TruncatedSVD(n_components=5,algorithm='randomized',n_iter=200,random_state=42)

In [8]:
tsvd_model.fit(dtm)

TruncatedSVD(algorithm='randomized', n_components=5, n_iter=200,
       random_state=42, tol=0.0)

# Busqueda de cuadricula
(Para hacer prueba con diferents valores en modelos y encontrar el mejor)

In [9]:
search_params = {'n_components':[5,10,15],'random_state':[42]}

In [11]:
tsvd = TruncatedSVD()

In [12]:
from sklearn.model_selection import GridSearchCV
model = GridSearchCV(tsvd,param_grid=search_params)

In [13]:
model.fit(dtm)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


TypeError: If no scoring is specified, the estimator passed should have a 'score' method. The estimator TruncatedSVD(algorithm='randomized', n_components=2, n_iter=5,
       random_state=None, tol=0.0) does not.

# Tema dominante en el texto

In [14]:
#Nombre de columnas
topicnames = []
for i in range(int(tsvd_model.n_components)):
    name='Topic'+str(i)
    topicnames.append(name)

In [15]:
#nombre de indices
docnames = []
#for i in range(len(cv.get_feature_names())):
for i in range(2225):
    docnames.append('Doc'+str(i))

In [16]:
import numpy as np
ica_output = tsvd_model.transform(dtm.toarray())
#Creando un dataframe en pandas
df_document_topic=pd.DataFrame(np.round(ica_output,2),columns=topicnames,index=docnames)

In [17]:
# Obtener tema dominante para cada documento
dominant_topic=np.argmax(df_document_topic.values,axis=1)
df_document_topic['dominant_topic']=dominant_topic

In [18]:
#Estilando
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

In [19]:
#Aplicando estilo
df_document_topic=df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topic

,Topic0,Topic1,Topic2,Topic3,Topic4,dominant_topic
Doc0,6.41,0.82,-3.86,-2.35,-1.91,0
Doc1,4.56,-0.34,-1.07,-1.66,0.38,0
Doc2,3.34,-0.83,-1.31,-1.47,-1.54,0
Doc3,6.63,0.02,-2.91,-2.87,-1.57,0
Doc4,2.56,-0.08,-1.35,-1.05,-0.76,0
Doc5,2.46,-0.19,-1.02,-0.64,-0.47,0
Doc6,5.27,-0.45,-2.08,-1.55,-1.28,0
Doc7,5.34,-1.27,-0.78,-1.77,-0.25,0
Doc8,2.89,0.3,-1.39,-0.79,-0.68,0
Doc9,1.72,-0.35,0.2,0.07,-0.22,0


# Palabras clave del tema

In [21]:
# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(tsvd_model.components_)

# Assign Column and Index
df_topic_keywords.columns = cv.get_feature_names()
df_topic_keywords.index = topicnames

# View
df_topic_keywords.head()

,00,000,000m,000th,007,01,0100,011,0130,02,...,zola,zombie,zombies,zone,zones,zoom,zooms,zurich,zvonareva,zvyagintsev
Topic0,0.000759,0.055270,0.001903,0.000106,0.000190,0.000328,0.000153,0.000072,0.000227,0.000072,...,0.000131,0.000383,0.000796,0.001159,0.000179,0.000170,0.000312,0.000672,0.000216,0.000039
Topic1,0.000520,0.006858,0.004554,0.000256,0.000750,0.000236,0.000365,-0.000010,0.000548,0.000161,...,0.000148,0.000766,-0.000215,-0.000077,0.000103,0.000262,0.000588,0.000466,0.000368,0.000143
Topic2,0.000308,-0.032342,-0.000275,-0.000005,0.000237,-0.000209,0.000064,-0.000125,0.000356,0.000011,...,-0.000180,-0.000067,-0.001381,-0.000937,-0.000110,-0.000471,-0.000616,-0.001120,-0.000230,0.000019
Topic3,0.000822,0.002656,-0.007886,-0.000266,-0.001129,-0.000813,-0.000289,-0.000177,-0.001715,-0.000324,...,-0.000459,0.000842,0.000170,-0.002076,-0.000016,0.000957,0.001237,-0.001468,-0.000921,-0.000334
Topic4,0.001572,-0.007200,0.007391,0.000032,0.000553,0.000282,-0.000446,-0.000074,-0.000499,0.000302,...,0.000495,-0.000577,-0.000646,-0.000092,0.000034,0.000112,-0.000791,0.001294,0.000672,0.000047


# 15 palabras clave principales de cada tema

In [22]:
#Obtener los temas correspondientes y las palabras relacionadas con cada tema
for index,topic in enumerate(tsvd_model.components_):
    print(f'The top 15 words for topic # {index}')
    print([cv.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print('\n')

The top 15 words for topic # 0
['told', 'world', 'just', 'like', 'labour', 'years', 'music', 'uk', 'time', 'government', 'year', 'new', 'people', 'mr', 'said']


The top 15 words for topic # 1
['robbie', 'think', 'game', 'year', 'urban', 'awards', 'like', 'people', 'won', 'angels', '25', 'years', 'song', 'music', 'best']


The top 15 words for topic # 2
['robbie', 'british', 'government', '25', 'angels', 'years', 'music', 'brown', 'party', 'election', 'blair', 'labour', 'song', 'best', 'mr']


The top 15 words for topic # 3
['black', 'tv', 'users', 'phones', 'services', 'use', 'like', 'digital', 'phone', 'technology', 'mr', 'urban', 'mobile', 'music', 'people']


The top 15 words for topic # 4
['nadal', 'gaming', 'half', 'ireland', 'playing', 'just', 'roddick', 'wales', 'play', 'england', 'world', 'new', 'time', 'games', 'game']




In [24]:
# Show top n keywords for each topic
def show_topics(cv=cv, tsvd_model=tsvd_model, n_words=20):
    keywords = np.array(cv.get_feature_names())
    topic_keywords = []
    for topic_weights in tsvd_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(cv=cv, tsvd_model=tsvd_model, n_words=15)        

# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,said,mr,people,new,year,government,time,uk,music,years,labour,like,just,world,told
Topic 1,best,music,song,years,25,angels,won,people,like,awards,urban,year,game,think,robbie
Topic 2,mr,best,song,labour,blair,election,party,brown,music,years,angels,25,government,british,robbie
Topic 3,people,music,mobile,urban,mr,technology,phone,digital,like,use,services,phones,users,tv,black
Topic 4,game,games,time,new,world,england,play,wales,roddick,just,playing,ireland,half,gaming,nadal


# SOM

In [25]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
from sompy.sompy import SOMFactory
from sklearn.datasets import fetch_california_housing

In [26]:
data = fetch_california_housing()
descr = data.DESCR
names = fetch_california_housing().feature_names+["HouseValue"]

data = np.column_stack([data.data, data.target])
print(descr)
print( "FEATURES: ", ", ".join(names))

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block
        - HouseAge      median house age in block
        - AveRooms      average number of rooms
        - AveBedrms     average number of bedrooms
        - Population    block population
        - AveOccup      average house occupancy
        - Latitude      house block latitude
        - Longitude     house block longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
http://lib.stat.cmu.edu/datasets/

The target variable is the median house value for California districts.

This dataset was derived from the 1990 U.S. census, using one row per census
block group. A block group is the smallest geographical unit for which the U.S.
Census Bur

In [ ]:
name=['Tech','Economy','Sport','Entertainment','Business']
#msz = calculate_msz(data)
sm = SOMFactory().build(ica_model.components_, normalization = 'var', initialization='random', component_names=name)
sm.train(n_job=-1, verbose=False, train_rough_len=2, train_finetune_len=5)

topographic_error = sm.calculate_topographic_error()
quantization_error = np.mean(sm._bmu[1])
print ("Topographic error = %s; Quantization error = %s" % (topographic_error, quantization_error))

In [ ]:
from sompy.visualization.mapview import View2D
view2D  = View2D(10,10,"rand data",text_size=10)
view2D.show(sm, col_sz=4, which_dim="all", denormalize=True)

In [ ]:
from sompy.visualization.bmuhits import BmuHitsView

vhts  = BmuHitsView(10,10,"Hits Map",text_size=7)
vhts.show(sm, anotate=True, onlyzeros=False, labelsize=10, cmap="autumn", logaritmic=False)
plt.show()
#cmap="Greys"

In [ ]:
#Kmeans cluster
from sompy.visualization.hitmap import HitMapView
sm.cluster(5)
hits  = HitMapView(10,10,"Clustering",text_size=7)
a=hits.show(sm, labelsize=12)